# Pulling Data from Google Earth Engine
Here, I try to:
- visualize the image I am trying to pull using Folium; and 
- export the image in the GeoTIFF format.

In [1]:
# Import packages
import ee
import pandas as pd
import numpy as np
import time
from geetools import batch
from tqdm import tqdm
from functools import partial
import folium
import rasterio as rio
from pyproj import Proj, transform
from osgeo import gdal, osr
import os
import geopandas as gpd

In [2]:
# Personal credentials
service_account = "sentinel2@uhi-causal-model.iam.gserviceaccount.com"
json_key = "../uhi-causal-model-a7e891b460a5.json"

In [3]:
# Initialize the library
ee.Initialize(ee.ServiceAccountCredentials(service_account, json_key), opt_url='https://earthengine-highvolume.googleapis.com')

In [4]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

## Recover the bounds from TIF file

The bounds will be extracted from the traversal data `Evening_Area-wide_Temperature_Boston.tif` to maintain across all data.

In [5]:
# Load the bounds from the Tiff file
path = "../data/traversal/Boston_ev.shp"
trav = gpd.read_file(path)

# Keeping the coordinate system consistent
trav = trav.to_crs(epsg=3857)
bounds = trav.total_bounds
print(f'Bounds in EPSG 3857 projection: {bounds}')

Bounds in EPSG 3857 projection: [-7922851.76390963  5197338.18061925 -7905185.54625323  5221224.54178838]


In [6]:
# Add a buffer to the bounds (in meters) to make sure that we have sufficient space around the traversal path.
# I recommend a buffer of 1000 meters, but to be conservative, you could make it 2000 meters, too.
print('before', bounds) 
new_bounds = [bounds[0]-1000, bounds[1]-1000, bounds[2]+1000, bounds[3]+1000]
print('after', new_bounds)

before [-7922851.76390963  5197338.18061925 -7905185.54625323  5221224.54178838]
after [-7923851.763909629, 5196338.180619254, -7904185.546253226, 5222224.541788379]


In [7]:
# Import dataset
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")

# Define parameters: temporal and spatial
startDate = '2019-07-29'
endDate = '2019-07-30'
lat,lon = 42.325347,-71.120802
crs = 'EPSG:3857'

# Define geometry
xMin, xMax, yMin, yMax = new_bounds[0], new_bounds[2], new_bounds[1], new_bounds[3]
geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]], crs, False, True)

# Get the single image
filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry)
image = filtered.median().clip(geometry)

## Folium Visualizations

### True color bands

In [15]:
# Visualizer
RGB_VIZ = {'min':0, 'max':255, 'bands':['TCI_R', 'TCI_G', 'TCI_B']}

# Display true color image
tci = image.select(['TCI_R', 'TCI_G', 'TCI_B'])
map = folium.Map(location=[lat, lon], zoom_start=11.4)
map.add_ee_layer(tci, RGB_VIZ, 'rgb')
display(map)

### NDVI

In [11]:
image2 = image.select(['B8', 'B4'])
ndvi = image2.normalizedDifference(['B8', 'B4'])

map = folium.Map(location=[lat, lon], zoom_start=11)
ndvi_params = {'min': -1, 'max': 1, 'palette': ['#640000','#ff0000','#ffff00','#00c800','#006400']}
map.add_ee_layer(ndvi, ndvi_params, 'NDVI')
display(map)

### SCL (raw band from Sentinel-2)

In [111]:
scl = image.select(['SCL'])
map = folium.Map(location=[lat, lon], zoom_start=11)
scl_params = {'min': 1, 'max': 11, 'bands': ['SCL'], 'palette':['#ff0004','#868686','#774b0a','#10d22c','#ffff52','#0000ff','#818181','#c0c0c0','#f1f1f1','#bac5eb','#52fff9']}
map.add_ee_layer(scl, scl_params, 'SCL')
display(map)

### Albedo

In [112]:
# this comes from the paper you reviewed on calculating albedo
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
weights = [0.2266, 0.1236, 0.1573, 0.3417, 0.1170, 0.0338]

# We can now calculate the albedo using this chain of functions
albedo = image.select(bands).multiply(ee.Image.constant(1e-4)).multiply(ee.Image.constant(weights)).reduce(ee.Reducer.sum())

In [113]:
map = folium.Map(location=[lat, lon], zoom_start=11)
albedo_params = {'min': 0, 'max': 0.4, 'palette': ['FF0000', '00FF00']}
map.add_ee_layer(albedo, albedo_params, 'albedo')
display(map)

## Exporting images

### NDVI

In [103]:
task1 = ee.batch.Export.image.toCloudStorage(
    image=ndvi,
    region=geometry,
    description="boston_ndvi",
    crs = crs,
    maxPixels=1e13,
    scale=10,
    bucket='earth-engine_boston',
    fileFormat='GeoTIFF'
)

task1.start()

In [109]:
# While the task is running, you can check on its status...it may take a few minutes
# to fully load the image.
task1.status()

{'state': 'COMPLETED',
 'description': 'boston_ndvi',
 'creation_timestamp_ms': 1708980682856,
 'update_timestamp_ms': 1708980842437,
 'start_timestamp_ms': 1708980688795,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://console.developers.google.com/storage/browser/earth-engine_boston/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 39.49863052368164,
 'id': 'AJTL27BAA5SN5YAQSVI7TQHL',
 'name': 'projects/earthengine-legacy/operations/AJTL27BAA5SN5YAQSVI7TQHL'}

### Albedo

In [29]:
task2 = ee.batch.Export.image.toCloudStorage(
    image=albedo,
    region=geometry,
    description="boston_albedo",
    crs = crs,
    maxPixels=1e13,
    scale=10,
    bucket='earth-engine_boston',
    fileFormat='GeoTIFF'
)

task2.start()

In [30]:
# While the task is running, you can check on its status...it may take a few minutes
# to fully load the image.
task2.status()

{'state': 'RUNNING',
 'description': 'boston_albedo',
 'creation_timestamp_ms': 1709213007247,
 'update_timestamp_ms': 1709213014103,
 'start_timestamp_ms': 1709213011092,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'RU2BJMS5CZR27ATP6KMEOCJC',
 'name': 'projects/earthengine-legacy/operations/RU2BJMS5CZR27ATP6KMEOCJC'}

### NLCD

In [100]:
# Input and output file paths
input_tiff_path = '../nlcd/nlcd_2021_land_cover_l48_20230630.img'
output_tiff_path = '../data/boston/boston_nlcd.tif'

# Bounds
te_options = ['-te', str(new_bounds[0]), str(new_bounds[1]), str(new_bounds[2]), str(new_bounds[3])]

# Target Spatial Reference System
t_srs_option = ['-t_srs', 'EPSG:3857']

# Target Resolution
tr_options = ['-tr', '30', '30']

# Options for Warping
options = ['-tap', '-of', 'GTiff']

# Combine all options
gdal_options = te_options + t_srs_option + tr_options + options

# Perform the Warp
gdal.Warp(output_tiff_path, input_tiff_path, options=gdal_options)

print("Warping completed successfully.")

Warping completed successfully.
